In [1]:
!pip install tensorflow

In [2]:
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.text import one_hot,Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.layers import Dense , Flatten ,Embedding,Input,LSTM, Bidirectional,GRU
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.preprocessing.text import text_to_word_sequence
from tensorflow.keras.initializers import Constant
from sklearn.preprocessing import LabelEncoder

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
%cd /content/drive/My Drive/project/

/content/drive/My Drive/project


In [7]:
true = pd.read_csv('Authentic-48K.csv', na_values=['#NAME?'])
false = pd.read_csv('Fake-1K.csv', na_values=['#NAME?'])

# Word Embeddings(word2vec)

In [8]:
corpus=[]
words = 0;
j = 0
for i in data['content'].values:
    corpus.append(str(i).split(" "))
    words += len(corpus[j])
    j += 1
print("Total words = ", words)
print("Total Sentances = ", len(corpus))

NameError: name 'data' is not defined

In [9]:
true = pd.read_csv('Authentic-48K.csv', na_values=['#NAME?'])
false = pd.read_csv('Fake-1K.csv', na_values=['#NAME?'])

# Concatenate the true and false DataFrames into a single DataFrame called 'data'
data = pd.concat([true, false], ignore_index=True)

corpus=[]
words = 0
j = 0
for i in data['content'].values:
    corpus.append(str(i).split(" "))
    words += len(corpus[j])
    j += 1
print("Total words = ", words)
print("Total Sentances = ", len(corpus))

Total words =  13873886
Total Sentances =  49977


In [10]:
from bnlp import BengaliWord2Vec
import gensim
from gensim.models import Word2Vec

bwv = BengaliWord2Vec()

trained_model_path = "./word_embeddings/bnwiki_word2vec.model"
data_file = corpus
model_name = "./word_embeddings/new_w2v.model"
vector_name = "./word_embeddings/new_w2v_vector.vector"
bwv.pretrain(trained_model_path, data_file, model_name, vector_name, epochs=10)


w2v_model = Word2Vec.load("./word_embeddings/new_w2v.model")
w2v_model.train(corpus,total_words=391114, epochs=10)

w2v_model.wv.most_similar('নষ্ট')
w2v_model.wv['বাংলাদেশ'].shape

ModuleNotFoundError: No module named 'bnlp'

In [11]:
!pip install bnlp_toolkit

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.9/58.9 kB 2.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.6/57.6 kB 2.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 175.4/175.4 kB 6.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.4/54.4 kB 3.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 26.7/26.7 MB 7.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 34.1/34.1 MB 16.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.4/78.4 kB 5.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 52.1 MB/s eta 0:00:00
  Created wheel for emoji: filename=emoji-1.7.0-py3-none-any.whl size=171033 sha256=1ec48be6f5dc1876f5dc83a3ce09290ff4715749d6c3c490c3e6c4f48d755b96
  Stored in directory: /root/.cache/pip/wheels/bd/22/e5/b69726d5e1a19795ecd3b3e7464b16c0f1d019aa94ff1c8578
Successfully built emoji
  Attempting uninstall: tqdm
    Found existin

In [12]:
from bnlp import BengaliWord2Vec # This line should work after installation
import gensim
from gensim.models import Word2Vec

# ... (Rest of the code remains the same)

punkt not found. downloading...


[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.


ImportError: cannot import name 'triu' from 'scipy.linalg' (/usr/local/lib/python3.11/dist-packages/scipy/linalg/__init__.py)

In [13]:
!pip install --upgrade scipy gensim bnlp_toolkit

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.0/62.0 kB 1.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.6/60.6 kB 3.4 MB/s eta 0:00:00


In [14]:
filename = './word_embeddings/fakeNews_word2vec_embeddings.txt'
w2v_model.wv.save_word2vec_format(filename, binary=False)

NameError: name 'w2v_model' is not defined

# Data processing for models and embedding matrix

In [ ]:
from sklearn.model_selection import train_test_split
from keras.utils import np_utils
import os

# tokenize every words so that evey words maps to numaric value
tok = Tokenizer()
tok.fit_on_texts(data['content'].values.astype('U'))
encd_rev = tok.texts_to_sequences(data['content'].values.astype('U'))
# tok.word_index

# make all the input sentance same length with add padding
max_rev_len = 40 # max lenght of a sentance
vocab_size = len(tok.word_index) + 1  # total no of words
embed_dim = 100 # embedding dimension
pad_rev= pad_sequences(encd_rev, maxlen=, padding='post')

print("Size of vocabulary : ", vocab_size)
print("All sentances with same length ", pad_rev.shape)

# lebel encode the output label to categorical
Y = data['label']
# encode class values as integers
encoder = LabelEncoder()
encoder.fit(Y)
encoded_Y = encoder.transform(Y)
# convert integers to dummy variables (i.e. one hot encoded)
dummy_y = np_utils.to_categorical(encoded_Y)

print(encoder.classes_)
print(dummy_y)

# test train split
X_train_word,X_test_word,y_train_word,y_test_word=train_test_split(pad_rev,dummy_y,test_size=0.30, random_state = 0)


# make a dictionary. word as key and feature vector as value
embedding_index={}
f = open('./word_embeddings/fakeNews_word2vec_embeddings.txt',encoding='utf-8')
for line in f:
    values=line.split()
    word=values[0]
    coefs=np.asarray(values[1:])
    embedding_index[word]=coefs
f.close()

# create a embeddings matrix with 100 dimenstion
EMBEDDING_DIM=100
embedding_matrix=np.zeros((vocab_size,EMBEDDING_DIM))
for word, i in tok.word_index.items():
    if i>vocab_size:
        continue
    embedding_vector=embedding_index.get(word)
    if embedding_vector is not None:
        embedding_matrix[i]=embedding_vector

Size of vocabulary :  52234
All sentances with same length  (2000, 300)
[0 1]
[[0. 1.]
 [0. 1.]
 [1. 0.]
 ...
 [1. 0.]
 [0. 1.]
 [1. 0.]]


# Model Training and Result evaluation helper function

In [ ]:
from sklearn.metrics import classification_report, confusion_matrix
def print_result(model, history, x_test, y_test, name):
    model_path = "./Models/"+ name+ " model"

    # convert the history.history dict to a pandas DataFrame:
    hist_df = pd.DataFrame(history.history)

    # save to json:
    hist_json_file = "./History/" + name + 'history.json'
    with open(hist_json_file, mode='w') as f:
        hist_df.to_json(f)

    #save model
    model.save(model_path)


    #ploting training  history
    accr = model.evaluate(x_test,y_test)
    print('Test set\n  Loss: {:0.3f}\n  Accuracy: {:0.3f}'.format(accr[0],accr[1]))

    y_pred = model.predict(x_test)

    #normalize
    y_pred_frac=np.argmax(y_pred, axis=1)
    y_test_frac=np.argmax(y_test, axis=1)

    print("-------------Classification Report----------------")
    print(classification_report(y_test_frac, y_pred_frac))
    print("-------------------//*//-------------------------")

    print("-------------Confusion Matrix----------------")
    cm = confusion_matrix(y_test_frac, y_pred_frac)
    print(cm)
    print("-------------------//*//-------------------------")



def train_model(model, X_train, y_train):
    epochs = 10
    batch_size = 64
    history = model.fit(X_train, y_train, epochs=epochs, batch_size=batch_size,validation_split=0.1)
    return history, model

# Bi-LSTM

In [ ]:
model=Sequential()
embedding_layer=Embedding(vocab_size,
                        EMBEDDING_DIM,
                        embeddings_initializer=Constant(embedding_matrix),
                        input_length=300,
                        trainable=False)
model.add(embedding_layer)
model.add(Bidirectional(LSTM(units=128,dropout=0.2,recurrent_dropout=0.2)))
model.add(Dense(2,activation='sigmoid'))
model.compile(loss='categorical_crossentropy',optimizer='adam',metrics=['accuracy'])
print('summary of the built model..')
model.summary()

summary of the built model..
Model: "sequential_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_4 (Embedding)     (None, 300, 100)          5223400   
                                                                 
 bidirectional_4 (Bidirectio  (None, 256)              234496    
 nal)                                                            
                                                                 
 dense_4 (Dense)             (None, 2)                 514       
                                                                 
Total params: 5,458,410
Trainable params: 235,010
Non-trainable params: 5,223,400
_________________________________________________________________


In [ ]:
history, model = train_model(model, X_train_word, y_train_word)

Epoch 1/10
20/20 [==============================] - 127s 6s/step - loss: 0.5390 - accuracy: 0.7325 - val_loss: 0.5069 - val_accuracy: 0.7714
Epoch 2/10
20/20 [==============================] - 124s 6s/step - loss: 0.3478 - accuracy: 0.8571 - val_loss: 0.3765 - val_accuracy: 0.8500
Epoch 3/10
20/20 [==============================] - 127s 6s/step - loss: 0.2611 - accuracy: 0.8944 - val_loss: 0.4659 - val_accuracy: 0.8214
Epoch 4/10
20/20 [==============================] - 128s 6s/step - loss: 0.2303 - accuracy: 0.9008 - val_loss: 0.4855 - val_accuracy: 0.8071
Epoch 5/10
20/20 [==============================] - 129s 6s/step - loss: 0.1876 - accuracy: 0.9246 - val_loss: 0.5113 - val_accuracy: 0.8286
Epoch 6/10
20/20 [==============================] - 129s 6s/step - loss: 0.1449 - accuracy: 0.9516 - val_loss: 0.5054 - val_accuracy: 0.8214
Epoch 7/10
20/20 [==============================] - 130s 7s/step - loss: 0.1085 - accuracy: 0.9643 - val_loss: 0.5051 - val_accuracy: 0.8143
Epoch 8/10
20

In [ ]:
print_result(model, history, X_test_word, y_test_word, 'LSTM')

INFO:tensorflow:Assets written to: ./Models/LSTM model\assets


19/19 [==============================] - 5s 220ms/step - loss: 0.6689 - accuracy: 0.6050
Test set
  Loss: 0.669
  Accuracy: 0.605
-------------Classification Report----------------
              precision    recall  f1-score   support

           0       0.60      0.67      0.63       306
           1       0.61      0.54      0.57       294

    accuracy                           0.60       600
   macro avg       0.61      0.60      0.60       600
weighted avg       0.61      0.60      0.60       600

-------------------//*//-------------------------
-------------Confusion Matrix----------------
[[204 102]
 [135 159]]
-------------------//*//-------------------------


# Bi-GRU

In [ ]:
model=Sequential()
embedding_layer=Embedding(vocab_size,
                        EMBEDDING_DIM,
                        embeddings_initializer=Constant(embedding_matrix),
                        input_length=300,
                        trainable=False)
model.add(embedding_layer)
model.add(Bidirectional(GRU(units=128,dropout=0.2,recurrent_dropout=0.2)))
model.add(Dense(2,activation='sigmoid'))
model.compile(loss='categorical_crossentropy',optimizer='adam',metrics=['accuracy'])
print('summary of the built model..')
model.summary()

summary of the built model..
Model: "sequential_6"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_6 (Embedding)     (None, 300, 100)          5223400   
                                                                 
 bidirectional_6 (Bidirectio  (None, 256)              176640    
 nal)                                                            
                                                                 
 dense_6 (Dense)             (None, 2)                 514       
                                                                 
Total params: 5,400,554
Trainable params: 177,154
Non-trainable params: 5,223,400
_________________________________________________________________


In [ ]:
history, model = train_model(model, X_train_word, y_train_word)

Epoch 1/10
20/20 [==============================] - 106s 5s/step - loss: 0.6203 - accuracy: 0.6579 - val_loss: 0.6021 - val_accuracy: 0.7214
Epoch 2/10
20/20 [==============================] - 97s 5s/step - loss: 0.4682 - accuracy: 0.7825 - val_loss: 0.5410 - val_accuracy: 0.7429
Epoch 3/10
20/20 [==============================] - 91s 5s/step - loss: 0.3839 - accuracy: 0.8294 - val_loss: 0.5110 - val_accuracy: 0.7929
Epoch 4/10
20/20 [==============================] - 90s 4s/step - loss: 0.3219 - accuracy: 0.8587 - val_loss: 0.4965 - val_accuracy: 0.8000
Epoch 5/10
20/20 [==============================] - 91s 5s/step - loss: 0.2653 - accuracy: 0.8937 - val_loss: 0.5156 - val_accuracy: 0.7643
Epoch 6/10
20/20 [==============================] - 92s 5s/step - loss: 0.2170 - accuracy: 0.9190 - val_loss: 0.4924 - val_accuracy: 0.8071
Epoch 7/10
20/20 [==============================] - 90s 5s/step - loss: 0.1763 - accuracy: 0.9429 - val_loss: 0.5146 - val_accuracy: 0.7929
Epoch 8/10
20/20 [=

In [ ]:
print_result(model, history, X_test_word, y_test_word, 'GRU')

INFO:tensorflow:Assets written to: ./Models/GRU model\assets


INFO:tensorflow:Assets written to: ./Models/GRU model\assets


19/19 [==============================] - 6s 340ms/step - loss: 0.4108 - accuracy: 0.8667
Test set
  Loss: 0.411
  Accuracy: 0.867
-------------Classification Report----------------
              precision    recall  f1-score   support

           0       0.90      0.83      0.86       306
           1       0.84      0.90      0.87       294

    accuracy                           0.87       600
   macro avg       0.87      0.87      0.87       600
weighted avg       0.87      0.87      0.87       600

-------------------//*//-------------------------
-------------Confusion Matrix----------------
[[255  51]
 [ 29 265]]
-------------------//*//-------------------------
